---

# <u> **The Naïve, Barnes-Hut, and Fast Multipole Methods**  </u>
### *Implementation, Analysis, and Evaluation of different Algorithms for $N$-Body Simulations*  

**Author:** Hugo Robijns
**Date:** April 2025  

---

### *Introduction:*

This report aims to analyse and implement two of these algorithms, the Barnes-Hut [CITE] and fast multipole method (FMM) [CITE] techniques, alongside the brute-force $\mathcal{O}(N^2)$ approach, in order to compare their performance and complexities. For the purpose of comparison, we will consider the 3D simulation of point masses under the influence of gravity.  


<center> Figure 1: a simulation of X </center>

---
## *1. Analysis of Algorithms*  


### 1.1 Naïve pairwise calculation, $\mathcal{O}(N^2)$

By Newton's Law of Gravitation, the gravitational force experienced by particle $i$ (mass $m_i$, position $\vec{r}_i$) due to a particle $j$ (mass $m_j$, position $\vec{r}_j$) is given by:
$$ \vec{F}_{ij} = \frac{Gm_im_j}{|\vec{r}_j-\vec{r}_i|^2}\cdot \frac{\vec{r}_j-\vec{r}_i}{|\vec{r}_j-\vec{r}_i|} = \frac{Gm_im_j (\vec{r}_j-\vec{r}_i)}{|\vec{r}_j-\vec{r}_i|^3} $$
&emsp; &emsp; therefore for an $N$-body system, the equations of motion become:
$$m_i \frac{\text{d}^2\vec{r}_i}{\text{d}t^2}  = \sum_{j=1, j\neq i}^{N} \frac{Gm_im_j (\vec{r}_j-\vec{r}_i)}{|\vec{r}_j-\vec{r}_i|^3}$$ 
It is clear to see that in order to simulate such a system directly, we must carry out $N-1$ calculations for each of the $N$ bodies (or half of the bodies, if exploiting Newton's $3^\text{rd}$ law), leading to a theoretical complexity of $\mathcal{O}(N^2)$. Note also that the force between two bodies diverges as they get close to each other: this requires the inclusion of a softening parameter $\epsilon$ which will be discussed further in section X.

### 1.2 Barnes-Hut Algorithm, $\mathcal{O}(N\log N)$  
The Barnes-Hut Algorithm attempts to reduce complexity by treating distance clusters of bodies as a single mass. It does this through the use of tree data structures, specifically an octree in 3D.



### 1.3 Fast Multipole Method (FMM)  
- Groups distant bodies into multipoles to reduce force calculations.  
- Computational complexity: **O(N)**  
- More complex but highly efficient for large-scale simulations. 

---

## 2. Implementation and Performance  

This section discusses how each method was implemented, including:  
- **Data structures used (e.g., trees, arrays, linked lists)**  
- **Optimization strategies (e.g., parallelization, caching, cutoff thresholds)**  
- **Performance metrics (e.g., execution time, memory usage, error analysis)**  

A performance comparison will be provided using **timing benchmarks** for different N.  

---

## 3. Results and Discussion  

This section presents the experimental results and evaluates the efficiency of each algorithm.  
- **Performance trade-offs**: How computational complexity translates into real execution time.  
- **Accuracy vs. Speed**: Comparing force approximations in Barnes-Hut and FMM.  
- **Scalability**: How well each method performs as N increases.  

Plots and tables will be included to illustrate findings.  
---

## Bibliography  

1. J. Barnes and P. Hut, *A Hierarchical O(N log N) Force-Calculation Algorithm*, Nature, 1986.  
2. L. Greengard and V. Rokhlin, *A Fast Algorithm for Particle Simulations*, Journal of Computational Physics, 1987.  
3. D. J. Griffiths, *Introduction to Electrodynamics*, Cambridge University Press, 2017.  